In [44]:
import numpy as np
import ee
import geemap
ee.Initialize()

In [12]:
import os
# os.listdir()

In [13]:
# The code generates monthly statistics from CHIRPS precipitation data.
# This includes:
# monthly total precip (mm)
# max storm each month (mm)
# median 5-day precipitation (mm)
# max 5-day precipitation (mm)
# sdii (total monthly precip / num storms >= 1 mm)

# The code generates a raster with 60 bands for each year. This includes 1
# band per month for each of the 5 summary statistics. The band names start 
# with a number, 0 for January up to 11 for December. The code is set to
# generate & export images for 2000 and 2016. You can add more images by
# duplicating the lines of code near the very bottom.

# The scale is set to 100km x 100km pixels. You may want to decrease the size.


In [60]:
cauvery = ee.FeatureCollection("projects/ee-gopalpenny/assets/india/cauvery_from_raster_250m_buffer_simplified")
# arkavathy = ee.FeatureCollection("users/gopalpenny/arkavathytanks/v_watershed_arkavathy_full")
boundary = cauvery
boundary_name = 'cauvery'

In [51]:

Map = geemap.Map()
Map.addLayer(boundary, {}, 'boundary')
Map.centerObject(boundary, 7)
Map

Map(center=[12.839298850454025, 77.44621600153913], controls=(WidgetControl(options=['position', 'transparent_…

In [15]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
chirps_pentad = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD")
era_land = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR")


In [61]:
start_year = 2016
end_year = 2022
years = ee.List.sequence(start_year,end_year)
yr = years.get(0)

def getChirps(monsoon_yr):
    monsoon_yr_start = ee.Date.fromYMD(monsoon_yr,6,1)
    monsoon_yr_end = monsoon_yr_start.advance(1, "year")
    chirps_monsoon = chirps_pentad.filterDate(monsoon_yr_start,monsoon_yr_end).reduce(ee.Reducer.sum()).rename('chirps')
    chirps_monsoon = chirps_monsoon.set('monsoon_yr',monsoon_yr)
    return chirps_monsoon

chirps_monsoon = ee.ImageCollection(years.map(getChirps))
chirps_monsoon.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'chirps',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'monsoon_yr': 2016, 'system:index': '0'}}

In [47]:
chirps_monsoon_list = chirps_monsoon.toList(100)

In [66]:
def download_chirps_monsoon_im(idx, boundary_name, folder):
    chirps_im = chirps_monsoon_list.get(idx)
    # print(idx)
    chirps_im = ee.Image(chirps_im)
    im_year = chirps_im.get('monsoon_yr').getInfo()
    im_name = boundary_name + '_chirps_monsoon_yr_' + str(im_year)
    chirps_task = ee.batch.Export.image.toDrive(image=chirps_im.int16(),
                                description=im_name,
                                folder=folder,
                                scale=5000,
                                region=boundary.geometry(),
                                maxPixels=1e13)
    print('Sending ' + im_name + ' to Google Drive')
    chirps_task.start()
    

In [80]:
for idx in np.arange(0,years.size().getInfo()):
    idx = int(idx) # ee doesn't like numby int64
    download_chirps_monsoon_im(idx, boundary_name, folder = 'CauveryGEE')



<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
